# Price Discovery Project Using Agents

This is a notebook that contains the experimentation of prioce discovery using agents.

## Import Libraries

In [29]:
import os
import re
import json
import pandas as pd
from typing import Any, Union


from PIL import Image

# from dotenv import load_dotenv


from qdrant_client import QdrantClient
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain.agents.agent import AgentAction
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.messages import HumanMessage
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.vectorstores import Chroma, Qdrant
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.tools.retriever import create_retriever_tool
from langchain_core.output_parsers import JsonOutputParser
from typing import Any, Optional, Type
from langchain.callbacks.manager import (
    CallbackManagerForToolRun,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser,
    OutputFixingParser,
)

## Specify Variables

In [30]:
# Load environment variables
# load_dotenv()

# Specify variables
model_name = "ViT-B-32"
checkpoint = "openai"
embedding_model = OpenCLIPEmbeddings(model_name=model_name, checkpoint=checkpoint)
# embedding_model_two = SentenceTransformerEmbeddings("clip-ViT-B-32")

embeddings = HuggingFaceEmbeddings(model_name="clip-ViT-B-32")

# db_dir = "db"

## Create Vector DB to store data

**Using Qdrant DataBase on Cloud**

In [31]:
os.environ["VECTORSTORE_API_KEY"] = ""

In [32]:
# Instantiate Client Variables
server_url = (
    "https://1883d972-3d1a-4e6d-a846-86b9952741cd.us-east4-0.gcp.cloud.qdrant.io/:6333"
)
qdrant_api_key = os.getenv("VECTORSTORE_API_KEY")

In [33]:
client = QdrantClient(server_url, api_key=qdrant_api_key)

text_doc_store = Qdrant(
    client=client,
    collection_name="apparel-collection",
    embeddings=embedding_model,
    content_payload_key="Record",
    metadata_payload_key="payload",
)

image_doc_store = Qdrant(
    client=client,
    collection_name="img-collection",
    embeddings=embedding_model,
    content_payload_key="Record",
    metadata_payload_key="payload",
)

In [34]:
query_vector = embedding_model.embed_query(
    "Women's Lightweight Open-Front Cardigan Sweater"
)

In [35]:
client.search("apparel-collection", query_vector=query_vector, limit=2)

[ScoredPoint(id='6a4bc5ca-7b13-433b-bac0-fab3c062e605', version=1428, score=0.9728498, payload={'Price': 28.5, 'asin': 'B07F2KS15P', 'title': "Women's Lightweight Open-Front Cardigan Sweater (Available in Plus Size)"}, vector=None, shard_key=None),
 ScoredPoint(id='2b327dd4-ab06-4979-95a4-213368813589', version=1479, score=0.95976985, payload={'Price': 24.72, 'asin': 'B07F28R9FW', 'title': "Women's Lightweight Longer Length Cardigan Sweater (Available in Plus Size)"}, vector=None, shard_key=None)]

## Create agent for retrieval

**Test data**

In [36]:
df = pd.read_csv("apparel_test.csv")
test_df = df.sample(n=3, random_state=329)
test_prices = test_df["price"].values
test_descriptions = test_df["title"].values
test_images = "apparel_images/" + test_df["imgName"].values

**Instatiate the llms**

In [37]:
# Mixtral
# Set up HuggingFace API

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

mixtral = HuggingFaceEndpoint(repo_id=repo_id, temperature=0.3)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\trung\.cache\huggingface\token
Login successful


In [38]:
# Setup google api
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

gemini = ChatGoogleGenerativeAI(
    model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0
)
gemini_vision = ChatGoogleGenerativeAI(
    model="gemini-pro-vision", google_api_key=GOOGLE_API_KEY, temperature=0.1
)

**Instantiate prompts**

In [39]:
min_price_schema = ResponseSchema(
    name="min_price",
    description="The reasonable minimum price for the product",
    type="number",
)
max_price_schema = ResponseSchema(
    name="max_price",
    description="The reasonable maximum price for the product",
    type="number",
)
reason_schema = ResponseSchema(
    name="reason", description="Reason for the price range", type="text"
)

price_range_parser = StructuredOutputParser.from_response_schemas(
    [min_price_schema, max_price_schema, reason_schema]
)

In [40]:
agent_prompt_template = """\
<s> [INST] As an experienced product analyst proficient in determining accurate product price ranges, utilize all available tools {tools}, including databases searches and internet searches, \
to precisely answer the given question. Begin by checking the database (there will always be product in the database), then proceed to search the internet if needed. \
Then pick the most similar products and use them to come up with an accurate price range.
Similar products are those with closely matching specifications based on criteria such as type of product, functionality, target users, style, material, and brand.

Based on this comparison, generate a highly accurate, reasonable, and compact estimate of the price range for the product \
and a comprehensive rationale for the specified price range. This should offer an in-depth understanding of how the determined \
price range was derived and why it aligns with the product's value proposition.
If a tool encounters an error, use another tool.

Please follow this format strictly:
Question: the input question you must answer.

Thought: you should always think about what to do.
Action: the action to take, should be ONE of or ALL of [{tool_names}]. Don't use any other tools.
Action Input: the input to the action. It SHOULD be a string.
Action: choose another tool if a tool fails or handle its error.
Observation: the result of the action, including insights gained or findings.
... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer that is based on the in-depth analysis of my findings and observations. If no result is found, use your own knowledge. (Always print this Thought before the final answer)
Final Answer: the final answer to the original input question, which must be as accurate and compact as possible. NEVER say you can't answer. {format_instructions}. 
Don't put any comments in the final answer.

Begin!

Question: {input}
Thought: {agent_scratchpad}
[/INST]</s>
"""

In [41]:
agent_prompt = PromptTemplate.from_template(
    agent_prompt_template,
    partial_variables={
        "format_instructions": price_range_parser.get_format_instructions()
    },
)

**Instantiate Agents**

In [42]:
# Instantiate error handler
class RetryCallbackHandler(BaseCallbackHandler):
    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        # Log the error or perform any necessary actions

        # Retry logic
        max_retries = 1
        current_retry = kwargs.get("retry_count", 0)
        if current_retry < max_retries:
            print(f"Retrying tool execution (Attempt {current_retry + 1})")
            # Increment retry count
            kwargs["retry_count"] = current_retry + 1
            # Re-run the tool
            return AgentAction.RERUN
        else:
            print("Maximum retries reached. Switching to another tool.")
            return AgentAction.CHANGE_TOOL

In [43]:
# The error handler
error_handler = RetryCallbackHandler()

In [44]:
# Create custom tools for retrieval


class Input(BaseModel):
    query: str = Field(description="should be a search query")


# Create variable context to store the retrieved data
context = []


class CustomTextRetrieverTool(BaseTool):
    name = "text db retriever"
    description = "useful for retrieving from a text database"
    args_schema: Type[BaseModel] = Input
    return_direct: bool = True

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        query_vector = embeddings.embed_query(query)
        query_results = client.search(
            "apparel-collection", query_vector=query_vector, limit=3
        )

        retrieved_data = ""
        for query_result in query_results:
            title = query_result.payload["title"]
            price = query_result.payload["Price"]

            context.append(title)

            retrieved_data += f"{title}. Price {price}\n"

        return retrieved_data


class CustomImageRetrieverTool(BaseTool):
    name = "image db retriever"
    description = "useful for retrieving from an image database"
    args_schema: Type[BaseModel] = Input
    return_direct: bool = True

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        query_vector = embeddings.embed_query(query)
        query_results = client.search(
            "img-collection", query_vector=query_vector, limit=3
        )

        retrieved_data = ""
        for query_result in query_results:
            title = query_result.payload["title"]
            price = query_result.payload["Price"]

            context.append(title)

            retrieved_data += f"{title}. Price {price}\n"

        return retrieved_data

In [45]:
# Function to extract description from image


def extract_description_from_image(description, image_path):
    image = Image.open(image_path)

    extract_description_from_image_prompt = HumanMessage(
        content=[
            {
                "type": "text",
                "text": f"""Your task is to generate a searchable prompt about the product in the image that I can feed to Google to find this exact product. You should:

1. Given the description {description}, extract relevant details about the product from the image, such as brand, color, and your thoughts on quality.

2. Use these details and the given description to generate a concise and well structured searchable prompt that would allow me to find this exact product on Google.

3. Return ONLY the searchable prompt in the following JSON format: {{"search_prompt": "<YOUR_SEARCHABLE_PROMPT>}}. Do not include any other information or text.

Please strictly follow these instructions and provide your response in the specified JSON format.""",
            },
            {
                "type": "image_url",
                "image_url": image,
            },
        ]
    )

    parser = JsonOutputParser()
    extract_description_from_image_chain = gemini_vision | parser
    response = extract_description_from_image_chain.invoke(
        [extract_description_from_image_prompt]
    )

    search_prompt = response["search_prompt"]

    return search_prompt

In [46]:
from huggingface_hub.utils import HfHubHTTPError
from langchain_core.exceptions import OutputParserException
import time


def parse_response(response, parser):
    try:
        parsed_response = parser.parse(response)
        return parsed_response
    except OutputParserException as e:
        print(e)
        print("Trying to fix the response format")
        try:
            output_fixing_prompt = PromptTemplate(
                input_variables=["completion", "error", "instructions"],
                template="<s> [INST] Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions: [/INST] </s>",
            )
            output_fixing_parser = OutputFixingParser.from_llm(
                parser=parser, prompt=output_fixing_prompt, llm=mixtral
            )
            parsed_response = output_fixing_parser.parse(response)
            return parsed_response
        except OutputParserException as e:
            print(e)
            print("Failed to fix the response format. Will send another LLM call")
            return -1
        except HfHubHTTPError as e:
            print(e)
            print(f"Slepping for 1 hour since {time.ctime()}")
            time.sleep(3600)


def get_response(chain, input={}):
    try:
        response = chain.invoke(input)
        return response
    except HfHubHTTPError as e:
        print(e)
        print(f"Slepping for 1 hour since {time.ctime()}")
        time.sleep(3600)


def get_llm_response(llm_chain, input={}, parser=None, max_tries=5):
    tries = 0
    while tries < max_tries:
        response = get_response(llm_chain, input)
        if parser is None:
            return response
        parsed_response = parse_response(response, parser)
        if parsed_response != -1:
            return parsed_response
        tries += 1
    raise Exception("Failed to parse the response")


def get_agent_response(agent_executor, input, parser=None, max_tries=5):
    tries = 0
    while tries < max_tries:
        agent_response = get_response(agent_executor, input)
        if parser is None:
            return agent_response
        parsed_response = parse_response(agent_response["output"], parser)
        if parsed_response != -1:
            return agent_response, parsed_response
        tries += 1
    raise Exception("Failed to parse the response")

In [47]:
# Function to extract title from internet search

title_schema = ResponseSchema(
    name="title", description="Only the title of the product.", type="text"
)
title_parser = StructuredOutputParser.from_response_schemas([title_schema])


def extract_context_from_internet_searches(agent_response):
    search_results = []
    for i in range(len(agent_response["intermediate_steps"])):
        cur_tool = agent_response["intermediate_steps"][i][0].tool
        if cur_tool == "search internet" or cur_tool == "search image":
            if type(agent_response["intermediate_steps"][i][1]) == list:
                search_results += agent_response["intermediate_steps"][i][1]
            else:
                search_results.append(agent_response["intermediate_steps"][i][1])

    extracted_contexts = []
    for search_result in search_results:
        if type(search_result) == dict and "title" in search_result:
            extracted_contexts.append(search_result["title"])
        else:
            extract_title_prompt = f"""<s>[INST]
            Extract exactly one product title from this search result.
            {str(search_result)}
            {title_parser.get_format_instructions()}

            Product Description:
            [/INST]</s>
            """
            extracted_title = get_llm_response(
                llm, input=extract_title_prompt, parser=title_parser
            )
            extracted_contexts.append(extracted_title["title"])

    return extracted_contexts

In [48]:
# Create tools
search = SerpAPIWrapper()
img_db_search = CustomImageRetrieverTool()
text_db_search = CustomTextRetrieverTool()

tool_search_item = Tool(
    name="search internet",
    description="Searches the internet for the price of a product by using its description.",
    func=search.run,
    handle_tool_error=True,
)


# Tool for retrieving product information from the text database based on the product description
# tool_retrieve_from_text_db = create_retriever_tool(
#     text_retriever,
#     'search text db',
#     'Query a retriever for product price using its product description.')

tool_image_retriever = Tool(
    name="search image db",
    description="Searches for the price of a product using its description in the image db",
    func=img_db_search.run,
    handle_tool_error=True,
)

tool_text_retriever = Tool(
    name="search text db",
    description="Searches for the price of a product using its description in the text db",
    func=text_db_search.run,
    handle_tool_error=True,
)
# List of all tools
tools = [tool_search_item, tool_image_retriever, tool_text_retriever]

In [49]:
serp_agent = create_react_agent(mixtral, tools, agent_prompt)


serp_agent_executor = AgentExecutor(
    agent=serp_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    callbacks=[error_handler],
)

In [50]:
extracted_descriptions = []
contexts = []
agent_responses = []
price_ranges = []
reasons = []

for i in range(len(test_descriptions)):
    test_description = test_descriptions[i]
    test_price = test_prices[i]
    test_image = test_images[i]

    print(i)
    print("Test Description: ", test_description)
    print("Test Price: $", test_price)

    extracted_description = extract_description_from_image(test_description, test_image)
    extracted_descriptions.append(extracted_description)

    context = []

    agent_response, parsed_response = get_agent_response(
        serp_agent_executor,
        {
            "input": f"what is a reasonable and accurate price range of product: {test_description} "
        },
        price_range_parser,
    )
    agent_responses.append(agent_response)
    price_ranges.append([parsed_response["min_price"], parsed_response["max_price"]])
    reasons.append(parsed_response["reason"])

    print("Price Range: ", [parsed_response["min_price"], parsed_response["max_price"]])

    extracted_contexts = extract_context_from_internet_searches(agent_response)
    context += extracted_contexts

    contexts.append(list(set(context)))

0
Test Description:  Anti-Theft Courier Saddle Crossbody, Stone, One Size
Test Price: $ 41.8




> Entering new AgentExecutor chain...
To answer this question, I will first search for the product in the text database and then, if necessary, search the internet for more information.

Action: search text db
Action Input: "Anti-Theft Courier Saddle Crossbody, Stone, One Size"

ObservationAnti-Theft Courier Saddle Crossbody, Stone, One Size. Price 41.8
Securtex® Anti-theft Free Time Crossbody Bag Crossbody. Price 97.4
Anti-Theft Metro Convertible Small Crossbody Bag. Price 39.99
I have found several similar products in the text database with prices ranging from $39.99 to $97.4. I will now analyze these findings to determine a reasonable price range for the Anti-Theft Courier Saddle Crossbody, Stone, One Size.

Final Answer:
```json
{
	"min_price": 39.99,
	"max_price": 97.4,
	"reason": "The Anti-Theft Courier Saddle Crossbody, Stone, One Size was found to have a price range of $39.99 to $97.4 in the text database. This price range is based on similar products with the same or similar

In [51]:
agent_dataset = {
    "test_descriptions": extracted_descriptions,
    "test_prices": test_prices,
    "agent_responses": agent_responses,
    "price_ranges": price_ranges,
    "reasons": reasons,
    "contexts": contexts,
}

In [52]:
agent_dataset

{'test_descriptions': ["Pacsafe Women's Anti-Theft Courier Crossbody Bag, Stone, One Size",
  "Women's Classic 11 M Aviator Sunglasses black metal frame grey gradient lenses",
  "Women's Wool Blend Hooded Cape Poncho Maxi Cloak Coat black"],
 'test_prices': array([ 41.8 , 275.  ,  69.99]),
 'agent_responses': [{'input': 'what is a reasonable and accurate price range of product: Anti-Theft Courier Saddle Crossbody, Stone, One Size ',
   'output': '```json\n{\n\t"min_price": 39.99,\n\t"max_price": 97.4,\n\t"reason": "The Anti-Theft Courier Saddle Crossbody, Stone, One Size was found to have a price range of $39.99 to $97.4 in the text database. This price range is based on similar products with the same or similar functionality, style, material, and brand."\n}\n```</s>',
   'intermediate_steps': [(AgentAction(tool='search text db', tool_input='Anti-Theft Courier Saddle Crossbody, Stone, One Size"\n\nObservation', log='To answer this question, I will first search for the product in the te

In [53]:
# import pickle
# with open('rag_eval/agent_dataset.pkl', 'wb') as f:
#     pickle.dump(agent_dataset, f)